## Lab 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=9428.92012, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 190.35996 each.], portfolio_value_time_series=[('2025-05-04 09:23:25', 9998.860120000001)])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 8857.840240000001, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 190.35996, "timestamp": "2025-05-04 09:23:25", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 190.35996, "timestamp": "2025-05-04 19:12:38", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-04 09:23:25", 9998.860120000001], ["2025-05-04 19:12:38", 9997.72024]], "total_portfolio_value": 9997.72024, "total_profit_loss": -2.2797600000012608}'

In [5]:
account.report()

'{"name": "ed", "balance": 8857.840240000001, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 190.35996, "timestamp": "2025-05-04 09:23:25", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 190.35996, "timestamp": "2025-05-04 19:12:38", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-04 09:23:25", 9998.860120000001], ["2025-05-04 19:12:38", 9997.72024], ["2025-05-04 19:12:39", 9997.72024]], "total_portfolio_value": 9997.72024, "total_profit_loss": -2.2797600000012608}'

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 190.35996,
  'timestamp': '2025-05-04 09:23:25',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 190.35996,
  'timestamp': '2025-05-04 19:12:38',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [7]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()


In [8]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [9]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4o-mini"

In [10]:


async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your current balance is $8,857.84. 

In terms of holdings, you own 6 shares of Amazon (AMZN).

### Now let's build our own MCP Client

In [11]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [12]:
agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
result = await Runner.run(agent, request)
display(Markdown(result.final_output))

Your account balance is **$8,857.84**. 

You hold **6 shares of Amazon (AMZN)**.

In [13]:
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 8857.840240000001, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 190.35996, "timestamp": "2025-05-04 09:23:25", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 190.35996, "timestamp": "2025-05-04 19:12:38", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-04 09:23:25", 9998.860120000001], ["2025-05-04 19:12:38", 9997.72024], ["2025-05-04 19:12:39", 9997.72024], ["2025-05-04 19:13:24", 9997.72024]], "total_portfolio_value": 9997.72024, "total_profit_loss": -2.2797600000012608}


In [14]:
from accounts import Account
Account.get("ed").report()

'{"name": "ed", "balance": 8857.840240000001, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 190.35996, "timestamp": "2025-05-04 09:23:25", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 190.35996, "timestamp": "2025-05-04 19:12:38", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-05-04 09:23:25", 9998.860120000001], ["2025-05-04 19:12:38", 9997.72024], ["2025-05-04 19:12:39", 9997.72024], ["2025-05-04 19:13:24", 9997.72024], ["2025-05-04 19:13:27", 9997.72024]], "total_portfolio_value": 9997.72024, "total_profit_loss": -2.2797600000012608}'